# Improvement of templates

## Static assertions (C++11)

The new `static_assert` directive allows you to check a condition **during compilation** and to display the message of your choice in case of failure. It is typically used to test a type used in a template:

In [ ]:
template <typename Int>
struct Rational {
    static_assert(sizeof(Int)>=4,"Underlying type size is not long enough\n") ;
    Int numerator ;
    Int denominator ;
} ; 

In [ ]:
Rational<int> r1 ;
Rational<short> r2 ;

In [ ]:
template <typename T, int size>
struct Array {
    static_assert(size>0,"Array size must be strictly positive\n") ;
    T data[size] ;
} ; 

In [ ]:
int const n { 2 } ;
// ...
Array<int,n> a1 ;
Array<int,n-1> a2 ;
Array<int,n-2> a3 ;

## Alias templates (C++11)

It becomes possible to define some kinds of `typedef` with template parameters.

The new keyword for this is `using`.

For example, it allows to give a name to a partial specialization of a class template:

In [ ]:
template <typename T, typename U>
class Pair { public : T x ; U y ; } ;

template <typename U>
using PairInt = Pair<int,U> ;

PairInt<double> pid ; // equivalent to Pair<int,double>

In [ ]:
template <typename T>
using MapInt = std::map<int,T> ;

MapInt<double> md ; // equivalent to std::map<int,double>

The `using` statement should now replace `typedef` under all circumstances. It is considered more readable. Below are some examples of equivalent instructions.

In [ ]:
#include <list>
#include <map>
#include <iostream>

In [ ]:
using real = float ; // typedef float real ;
    
using my_map = std::map<std::vector<int>,std::list<float>> ;
using my_citr = my_map::const_iterator ;
    
using fptr = void(*)(int) ;

Starting from C ++ 14, the standard library includes some **`_t`** shortcuts for the templates that are used to manipulate on types. For example:

In [ ]:
template<typename T>
using remove_pointer_t = typename std::remove_pointer<T>::type ;

Note however that with the appearance of `auto`, it is less and less necessary to use a type alias to lighten the writing of user code.

## Variadic templates (C++11)

C ++ 11 introduces the possibility of defining templates with a variable number of parameters.

#### Pass-through functions

A simple use of `typename...` is useful to take a pack of arguments and transmit them to another function.

In [ ]:
#include <iostream>
#include <string>

template < typename Function, typename... Args >
void apply( std::string const & comment, Function f, Args... args )
 {
  std::cout<<"("<<comment<<") " ;
  f(args...) ;
 }  

In [ ]:
void print( int i, double d )
 { std::cout<<i<<" "<<d<<std::endl ; }  

In [ ]:
apply("printing",print,42,3.14) ;

#### Looping on template fonction parameters

When one want to loop over all the parameters, each of them with a type potentially different. With functions, you can rely on template functions which call one another recursively, and let overload resolution process:

In [ ]:
#include <iostream>

template <typename T>
void print( T last ) {
  std::cout<<last<<std::endl  ;
}

template <typename T, typename... Types>
void print( T first, Types... others ) {
  std::cout<<first<<" "  ;
  print(others...) ;
 }

In [ ]:
print("(printing)",42,3.14) ;

#### Looping on template class parameters

It gets more delicate when one want to loop the parameters of a template class. This requires a **recursive partial specialization**.

In [ ]:
%%file tmp.tuples.h

template <typename... Types> struct Tuple ;

template <typename T, typename... Types>
struct Tuple<T, Types...>
 {
  T data ;
  Tuple<Types...> others ;
 } ;

template <> struct Tuple<> {} ;

In [ ]:
%%file tmp.templates.cpp

#include <iostream>
#include <string>
#include "tmp.tuples.h"

int main()
 {
  Tuple<int,double,std::string> tuple { 42, 3.14, "bonjour" } ;
  std::cout << tuple.data << std::endl ;
  std::cout << tuple.others.data << std::endl ;
  std::cout << tuple.others.others.data << std::endl ;
 }

In [ ]:
!rm -f tmp.templates.exe && g++ -std=c++17 tmp.templates.cpp -o tmp.templates.exe

In [ ]:
!./tmp.templates.exe

## Variable templates (C++14)

C++14 introduces the possibility of making template of variables. The example usually given is a constant variable pi, that we could define with different precision for all the predefined types.

In [ ]:
%%file tmp.templates.cpp

#include <iostream>

template<typename T>
const T pi = T(3.1415926535897932385) ;

template<typename T>
T circular_area(T a_r)
 { return pi<T> * a_r * a_r ; }

int main()
 {
  std::cout.precision(18) ;
  std::cout << "double : " << circular_area(1.) << std::endl ;
  std::cout << "float  : " << circular_area(1.f) << std::endl ;
  std::cout << "int    : " << circular_area(1) << std::endl ;
 }

In [ ]:
!rm -f tmp.templates.exe && g++ -std=c++17 tmp.templates.cpp -o tmp.templates.exe

In [ ]:
!./tmp.templates.exe

Starting from C++17, the standard library includes some **`_v` shortcuts** for the templates that are testing type properties. For example:

In [ ]:
%%file tmp.templates.cpp

#include <type_traits>

template< class T >
bool const is_integral_v = std::is_integral<T>::value ;

template <typename IntegralT>
struct Rational {
    static_assert(is_integral_v<IntegralT>,"Bad IntegralT") ;
    IntegralT numerator ;
    IntegralT denominator ;
} ;

int main() {
  Rational<int> r1 ;
  Rational<double> r2 ;
}

In [ ]:
!rm -f tmp.templates.exe && g++ -std=c++17 tmp.templates.cpp -o tmp.templates.exe

## Class template argument deduction (C++17)

For a template function, the compiler was already able to infer the template parameters from the type of runtime arguments. Starting with C++17, it is also okay to work with classes.

In [ ]:
template <typename Int>
struct Rational {
    Rational( Int n, Int d ) : numerator(n), denominator(d) {}
    Int numerator ;
    Int denominator ;
} ; 

In [ ]:
Rational r { 1, 2 } ;

The deduction rely on implicitly-generated deduction guides. They are sometimes insufficient:

In [ ]:
%%file tmp.ctad.cpp

template <typename Int>
struct Rational {
    Int numerator ;
    Int denominator ;
} ; 

int main() {
  Rational<int> r1 { 1, 2 } ;
  Rational r2 { 1, 2 } ;
}

In [ ]:
!rm -f tmp.ctad.exe && g++ -std=c++17 tmp.ctad.cpp -o tmp.ctad.exe

In such a case, one can help the compiler by adding *user-defined deduction guides*:

In [ ]:
%%file tmp.ctad.cpp

template <typename Int>
struct Rational {
    Int numerator ;
    Int denominator ;
} ; 

// user-defined deduction guide
template<class Int>
Rational(Int n, Int m) -> Rational<Int> ;

int main() {
  Rational r2 { 1, 2 } ;
}

In [ ]:
!rm -f tmp.ctad.exe && g++ -std=c++17 tmp.ctad.cpp -o tmp.ctad.exe

## Concepts (C++20)

The long awaited way to declare constraints on template parameters.

In [ ]:
%%file tmp.concepts.cpp

#include <type_traits>

template< typename T >
concept integral = std::is_integral_v<T>;

template <typename Int>
requires integral<Int>
struct Rational {
    Int numerator ;
    Int denominator ;
} ; 

int main() {
  Rational<double> r2 { 1., 2. } ;
}

In [ ]:
!rm -f tmp.concepts.exe && g++ -std=c++20 tmp.concepts.cpp -o tmp.concepts.exe

# Take away

- Each new version of C++ introduces some new template syntax, which is generally applied to the standard  library in the next version:
  - Before, there was `std::numeric_limits<T>::is_exact`,
  - C++11 library introduced `std::is_floating_point<T>::value`,
  - C++14 syntax introduced variable templates,
  - C++17 library introduced `std::is_floating_point_v<T>`,
  - C++20 introduced concepts and `std::floating_point<T>`.
- The C++20 concepts is a real game changer, especially for the libraries authors. New libraries will now generally require C++20.

# Questions ?

# Exercise

Complete the `make_ptr` function, imitation of `std::make_shared`.

In [ ]:
%%file tmp.templates.cpp

#include <memory>
#include <iostream>

class MyData
 {
  public :
    MyData( int i, double d ) : m_i(i), m_d(d)
     { std::cout<<"MyData::MyData()"<<std::endl ; }
    int i() { return m_i ; }
    double d() { return m_d ; }
    ~MyData() { std::cout<<"MyData::~MyData()"<<std::endl ; }
  private :
    int m_i ;
    double m_d ;
 } ;

void display( std::shared_ptr<MyData> data_ptr )
 { std::cout<<data_ptr->i()<<" "<<data_ptr->d()<<std::endl ; }  

template <typename T, typename... Args>
std::shared_ptr<T> make_ptr( Args... args )
 { return ??? ; }  

int main()
 {
  auto data_ptr {make_ptr<MyData>(42,3.14)} ;
  display(data_ptr) ;
  return 0 ;
 }


In [ ]:
!rm -f tmp.templates.exe && g++ -std=c++17 tmp.templates.cpp -o tmp.templates.exe

In [ ]:
!./tmp.templates.exe

# Resources

- [CTAD](https://en.cppreference.com/w/cpp/language/class_template_argument_deduction)


© *CNRS 2024*  
*This document was created by David Chamont and translated by Olga Abramkina. It is available under the [License Creative Commons - Attribution - No commercial use - Shared under the conditions 4.0 International](http://creativecommons.org/licenses/by-nc-sa/4.0/)*